In [58]:
import pandas as pd
import requests

# URL is only using central Sierra Nevada Monitoring station with Measurment 18 Daily Snow Depth, Some Stations can be dropped if they do not have '18' Daily snow depth, there is a warning on the website that says "if the station doesn't use the measurment it will be dropped automatically"
# Ajusted data to start date to be October 1st in 2012 to allow us to have 10 winter seasons Snow Depth measurments to pair with 10 full fire seasons
# Dropped DDM station due to several readings of snow pack over 2k, days before and after the reading showed significantly lower snowpack
url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ALP%2CBLC%2CBLK%2CBLS%2CBSK%2CCAP%2CCSL%2CDAN%2CEBB%2CEP5%2CFDC%2CFLL%2CGIN%2CGKS%2CGNL%2CHGM%2CHOR%2CHRS%2CHVN%2CHYS%2CLBD%2CLVM%2CLVT%2CMDW%2CMNT%2CMRL%2CMSK%2CPSN%2CRBB%2CRBP%2CRP2%2CSDW%2CSIL%2CSLI%2CSPS%2CSPT%2CSTR%2CTCC%2CTNY%2CTUM%2CVRG%2CWC3%2CWHW&SensorNums=18&dur_code=D&Start=2012-10-01&End=2023-04-14"
response = requests.get(url).json()

In [59]:
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year
#Clipped off negative values that don't make sense for snow depth, (shown as -9999 by monitoring stations)
snow_df['Snow Depth (in)'] = snow_df['Snow Depth (in)'].clip(lower = 0)
# snow_df['Snow Depth (in)'].min()

In [60]:
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2013

for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year - 1} {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1

snow_season["Season"] = season_list

In [61]:
SS_list =snow_season.groupby(['Season'])
SS_list.head()

,Date,Station ID,Snow Depth (in),Month,Year,Season
0,2012-10-01,BLC,0,10,2012,Summer 2012 2013
1,2012-10-02,BLC,1,10,2012,Summer 2012 2013
2,2012-10-03,BLC,0,10,2012,Summer 2012 2013
3,2012-10-04,BLC,1,10,2012,Summer 2012 2013
4,2012-10-05,BLC,1,10,2012,Summer 2012 2013
...,...,...,...,...,...,...
3537,2023-01-24,BLC,71,1,2023,Winter 2022 2023
3538,2023-01-25,BLC,69,1,2023,Winter 2022 2023
3539,2023-01-26,BLC,68,1,2023,Winter 2022 2023
3540,2023-01-27,BLC,67,1,2023,Winter 2022 2023


In [62]:
# snow_season['Mean Snow Depth (in)'] = snow_season.groupby(snow_season['Snow Depth (in)'].mean())['Station ID']
# snow_season['Max Snow Depth (in)'] = snow_season.groupby(['Snow Depth (in)']).max()
# snow_season

In [65]:
winter_only_mask = snow_season["Season"].str.contains("Winter")
snow_season_winter_only = snow_season.loc[winter_only_mask]
snow_season_grouped = snow_season_winter_only.groupby(["Season"])
snow_season_grouped.head(2)

,Date,Station ID,Snow Depth (in),Month,Year,Season
29,2012-11-01,BLC,3,11,2012,Winter 2012 2013
30,2012-11-02,BLC,2,11,2012,Winter 2012 2013
433,2014-01-02,BLC,9,1,2014,Winter 2013 2014
434,2014-01-03,BLC,9,1,2014,Winter 2013 2014
789,2015-01-02,BLC,13,1,2015,Winter 2014 2015
790,2015-01-03,BLC,13,1,2015,Winter 2014 2015
1151,2016-01-02,BLC,22,1,2016,Winter 2015 2016
1152,2016-01-03,BLC,21,1,2016,Winter 2015 2016
1506,2017-01-02,BLC,16,1,2017,Winter 2016 2017
1507,2017-01-04,BLC,38,1,2017,Winter 2016 2017
